### 치매 자가 진단을 위한 인솔 보행 데이터 기반 분석 서비스

<br>

[치매 고위험군 웨어러블 라이프로그](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=226)
<br>[치매 자가 진단을 위한 인솔 보행 데이터 기반 분석 서비스](https://aifactory.space/task/2638/overview)

<br>

In [1]:
import numpy as np
import pandas as pd
import sys

import torch
import torch.nn as nn
import torch.optim as optim

In [7]:
x_train_path = 'data/x_train.npy'
y_train_path = 'data/y_train.npy'
model_save_path = 'output/pytorch_model.bin'

x_train = pd.DataFrame(np.load(x_train_path))
y_train = pd.DataFrame(np.load(y_train_path))

### ---------------------- 데이터 윈도윙 ----------------------

data = x_train.values # numpy 배열로 변경
window_size = 10 # 슬라이딩 윈도우 크기
windows = [] # 슬라이딩 윈도우 적용

for i in range(data.shape[0] - window_size + 1) : # 윈도윙
    windows.append(data[i :i + window_size])

windows = np.array(windows) # 결과를 numpy 배열로 변환

### ---------------------- 윈도우별 레이블 추가 ----------------------

is0 = 0
is10 = 0
is20 = 0
is30 = 0
Y = []
for i in range(data.shape[0] - window_size + 1) :
    if windows[i].sum(axis = 0)[25] == 0:
        is0 = is0 + 1
        Y.append([0])
    elif windows[i].sum(axis = 0)[25] == 10:
        is10 = is10 + 1
        Y.append([1])
    elif windows[i].sum(axis = 0)[25] == 20:
        is20 = is20 + 1
        Y.append([2])
    elif windows[i].sum(axis = 0)[25] == 30 :
        is30 = is30 + 1
        Y.append([3])
    else :
        Y.append([4])


### ---------------------- Make Model ----------------------

class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(128, 128)
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(128, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

### ---------------------- Train Setting ----------------------

# 하이퍼파라미터 설정
Window_size = 10
batch_size = 32
input_size = 10 * 25
output_size = 1
learning_rate = 0.001
num_epochs = 1

device = torch.device("cuda")

X = torch.tensor(windows[:, :, :-1], dtype = torch.float32)
Y = torch.tensor(np.array(Y), dtype = torch.float32)

dataset = torch.utils.data.TensorDataset(X, Y)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = MLP(input_size, output_size)
model.to(device)

criterion = nn.BCELoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습
correct = 0
total = 0
for epoch in range(num_epochs):
    for batch_X, batch_Y in dataloader:
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)
        optimizer.zero_grad()  # 그래디언트 초기화
        outputs = model(batch_X)  # 모델 예측

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_Y).sum()
        total += len(batch_Y)
        
        loss = criterion(outputs, batch_Y.float())  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 옵티마이저 업데이트

    accuracy = round((correct.float() / total).item(), 4)     
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Accuracy: {accuracy}')

torch.save(model.state_dict(), model_save_path)


Epoch [1/10], Loss: 0.0, Accuracy: 32.0
Epoch [2/10], Loss: 0.0, Accuracy: 32.0
Epoch [3/10], Loss: 0.0, Accuracy: 32.0
Epoch [4/10], Loss: 0.0, Accuracy: 32.0
Epoch [5/10], Loss: 0.0, Accuracy: 32.0
Epoch [6/10], Loss: 0.0, Accuracy: 32.0
Epoch [7/10], Loss: 0.0, Accuracy: 32.0
Epoch [8/10], Loss: 0.0, Accuracy: 32.0
Epoch [9/10], Loss: 0.0, Accuracy: 32.0
Epoch [10/10], Loss: 0.0, Accuracy: 32.0


In [9]:
import numpy as np
import sys

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cpu")
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(128, 128)
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(128, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x
    
x_test_path = 'data/x_train.npy'
y_pred_save_path = 'y_pred.npy'
model_path = 'output/pytorch_model.bin'

input_size = 10 * 25
output_size = 1
learning_rate = 0.001

x_test = np.load(x_test_path)
a = pd.DataFrame(x_test)
x_test = np.array(a)

window_size = 10 # 슬라이딩 윈도우 크기
windows = [] # 슬라이딩 윈도우 적용
Y = []

for i in range(x_test.shape[0] - window_size + 1) : # 윈도윙
    windows.append(x_test[i :i + window_size])


x_test = np.array(windows) # 결과를 numpy 배열로 변환

is0 = 0
is10 = 0
is20 = 0
is30 = 0
for i in range(x_test.shape[0] - window_size + 1) :
    if windows[i].sum(axis = 0)[25] == 0:
        is0 = is0 + 1
        Y.append([0])
    elif windows[i].sum(axis = 0)[25] == 10:
        is10 = is10 + 1
        Y.append([1])
    elif windows[i].sum(axis = 0)[25] == 20:
        is20 = is20 + 1
        Y.append([2])
    elif windows[i].sum(axis = 0)[25] == 30 :
        is30 = is30 + 1
        Y.append([3])
    else :
        Y.append([4])

for i in range(9):
    Y.append([0])


# print(f'Y : {len(Y)}')
# print(f'x_test : {x_test.shape}')

x_test = torch.tensor(x_test[:, :, :-1], dtype = torch.float32)
Y = torch.tensor(np.array(Y), dtype = torch.float32)

dataset = torch.utils.data.TensorDataset(x_test, Y)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

model = MLP(input_size, output_size)
model.load_state_dict(torch.load(model_path))
model.to(device)

criterion = nn.BCELoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

y_pred = []
for batch_X, batch_Y in dataloader :
    batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)
    y_pred.append(model(batch_X).detach().numpy())
# print(f'y_pred : {y_pred}')
# y_pred_labels = np.argmax(y_pred, axis=1)
y_pred = np.array(y_pred)
np.save(y_pred_save_path, y_pred)
